# Import Libraries

In [55]:
import numpy as np
import math
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

from pyDOE import lhs         #Latin Hypercube Sampling

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

In [154]:
layers = np.array([2, 50, 50, 50, 1]) #3 hidden layers

W = []  #Weights and biases
parameters = 0 #total number of parameters

for i in range(len(layers)-1):

    input_dim = layers[i]
    output_dim = layers[i+1]

    #Xavier standard deviation 
    std_dv = np.sqrt((2.0/(input_dim + output_dim)))

    #weights = normal distribution * Xavier standard deviation + 0
    w = torch.normal(0,1,size=(input_dim, output_dim),requires_grad=True) * std_dv

    b = torch.zeros((output_dim,1),requires_grad=True)

    print(b.shape)
    
    W.append(w)
    W.append(b)

    parameters +=  input_dim * output_dim + output_dim
    
N_u = 400 #Total number of data points for 'u'
N_f = 10000 #Total number of collocation points 

# Training data
X_f_train, X_u_train, u_train = trainingdata(N_u,N_f)

x = X_u_test
x = torch.rand_like(torch.from_numpy(x), dtype=torch.float64)
output = m(x)

c = output.size

print(list(c.size()))

torch.Size([50, 1])
torch.Size([50, 1])
torch.Size([50, 1])
torch.Size([1, 1])


AttributeError: 'builtin_function_or_method' object has no attribute 'size'

# *Data Prep*

Training and Testing data is prepared from the solution file

In [57]:
x_1 = np.linspace(-1,1,256)  # 256 points between -1 and 1 [256x1]
x_2 = np.linspace(1,-1,256)  # 256 points between 1 and -1 [256x1]

X, Y = np.meshgrid(x_1,x_2) 

# Test Data

We prepare the test data to compare against the solution produced by the PINN.

In [58]:
X_u_test = np.hstack((X.flatten(order='F')[:,None], Y.flatten(order='F')[:,None]))

# Domain bounds
lb = np.array([-1, -1]) #lower bound
ub = np.array([1, 1])  #upper bound

a_1 = 1 
a_2 = 1

usol = np.sin(a_1 * np.pi * X) * np.sin(a_2 * np.pi * Y) #solution chosen for convinience  

u = usol.flatten('F')[:,None] 

# Training Data

In [59]:
def trainingdata(N_u,N_f):
    
    leftedge_x = np.hstack((X[:,0][:,None], Y[:,0][:,None]))
    leftedge_u = usol[:,0][:,None]
    
    rightedge_x = np.hstack((X[:,-1][:,None], Y[:,-1][:,None]))
    rightedge_u = usol[:,-1][:,None]
    
    topedge_x = np.hstack((X[0,:][:,None], Y[0,:][:,None]))
    topedge_u = usol[0,:][:,None]
    
    bottomedge_x = np.hstack((X[-1,:][:,None], Y[-1,:][:,None]))
    bottomedge_u = usol[-1,:][:,None]
    
    all_X_u_train = np.vstack([leftedge_x, rightedge_x, bottomedge_x, topedge_x])
    all_u_train = np.vstack([leftedge_u, rightedge_u, bottomedge_u, topedge_u])  
     
    #choose random N_u points for training
    idx = np.random.choice(all_X_u_train.shape[0], N_u, replace=False) 
    
    X_u_train = all_X_u_train[idx[0:N_u], :] #choose indices from  set 'idx' (x,t)
    u_train = all_u_train[idx[0:N_u],:]      #choose corresponding u
    
    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    X_f = lb + (ub-lb)*lhs(2,N_f) 
    X_f_train = np.vstack((X_f, X_u_train)) # append training points to collocation points 
    
    return X_f_train, X_u_train, u_train 


# Physics Informed Neural Networks

In [161]:
class Sequentialmodel():
    
    def __init__(self,layers):

        self.W = []  #Weights and biases
        self.parameters = 0 #total number of parameters
        self.activation = nn.Tanh()
        self.loss = nn.MSELoss(reduction ='mean')

        for i in range(len(layers)-1):
            
            input_dim = layers[i]
            output_dim = layers[i+1]
            
            #Xavier standard deviation 
            std_dv = np.sqrt((2.0/(input_dim + output_dim)))
            
            #weights = normal distribution * Xavier standard deviation + 0
            w = torch.normal(0,1,size=(input_dim, output_dim),requires_grad=True, dtype=torch.float64) * std_dv
            
            b = torch.zeros((output_dim),requires_grad=True)
                       
            self.W.append(w)
            self.W.append(b)

            self.parameters +=  input_dim * output_dim + output_dim
            
            
    def forward(self,x):
        
        x = torch.rand_like(torch.from_numpy(x), dtype=torch.float64)
                
        #preprocessing input 
        x = (x - lb)/(ub - lb) #feature scaling
        
        a = x
        
        for i in range(len(layers)-2):
            
            z = a.matmul(self.W[2*i]) + self.W[2*i+1]
                        
            a = self.activation(z)
            
        a = a.matmul(self.W[-2]) + self.W[-1]
        
        return a
      
#     def get_weights(self):

#         parameters_1d = []  # [.... W_i,b_i.....  ] 1d array

#         for i in range (len(layers)-1):
            
#             w_1d = W[2*i].view(-1)
#             b_1d = W[2*i+1].view(-1)
            
#             parameters_1d = torch.cat([parameters_1d,w_1d],0)
#             parameters_1d = torch.cat([parameters_1d,b_1d],0)
        
#         return parameters_1d   
    
    
#     def set_weights(self,parameters):
                
#         for i in range (len(layers)-1):
            
#             shape_w = W[2*i].shape
            
#             tf.shape(self.W[2*i]).numpy() # shape of the weight tensor
#             size_w = tf.size(self.W[2*i]).numpy() #size of the weight tensor 
            
#             shape_b = tf.shape(self.W[2*i+1]).numpy() # shape of the bias tensor
#             size_b = tf.size(self.W[2*i+1]).numpy() #size of the bias tensor 
            
                        
    def loss_BC(self,x,y):
        
        x = torch.rand_like(torch.from_numpy(x), dtype=torch.float64)
        
        y = torch.rand_like(torch.from_numpy(y), dtype=torch.float64)
        
        loss_u = self.loss(x, y)
        
        return loss_u
    
    def loss_PDE(self, x_to_train_f):
        
        
        
        
        return loss_f, f
    
    def loss(self,x,y,g):

        loss_u = self.loss_BC(x,y)
        loss_f, f = self.loss_PDE(g)

        loss = loss_u + loss_f

        return loss, loss_u, loss_f 
     
        
#     def optimizerfunc(self,parameters):
        
#         return loss_val.numpy(), grads_1d.numpy()

In [163]:
N_u = 400 #Total number of data points for 'u'
N_f = 10000 #Total number of collocation points 

# Training data
X_f_train, X_u_train, u_train = trainingdata(N_u,N_f)

layers = np.array([2, 50, 50, 50, 1]) #3 hidden layers

maxcor = 200 
max_iter = 5000

PINN = Sequentialmodel(layers)

u_pred = PINN.forward(X_u_test)

l_bc = PINN.loss_BC(X_u_train, X_u_train)

print(l_bc)

tensor(0.1690, dtype=torch.float64)


# Main

In [ ]:
N_u = 400 #Total number of data points for 'u'
N_f = 10000 #Total number of collocation points 

# Training data
X_f_train, X_u_train, u_train = trainingdata(N_u,N_f)

layers = np.array([2, 50, 50, 50, 1]) #3 hidden layers

maxcor = 200 
max_iter = 5000

PINN = Sequentialmodel(layers)

init_params = PINN.get_weights().numpy()

start_time = time.time()

elapsed = time.time() - start_time                
print('Training time: %.2f' % (elapsed))

# optimization 

''' Model Accuracy ''' 
u_pred = PINN.evaluate(X_u_test)

error_vec = np.linalg.norm((u-u_pred),2)/np.linalg.norm(u,2)        # Relative L2 Norm of the error (Vector)
print('Test Error: %.5f'  % (error_vec))

u_pred = np.reshape(u_pred,(256,256),order='F') 


# Plotting

#Ground truth
fig_1 = plt.figure(1, figsize=(18, 5))
plt.subplot(1, 3, 1)
plt.pcolor(x_1, x_2, usol, cmap='jet')
plt.colorbar()
plt.xlabel(r'$x_1$', fontsize=18)
plt.ylabel(r'$x_2$', fontsize=18)
plt.title('Ground Truth $u(x_1,x_2)$', fontsize=15)

# Prediction
plt.subplot(1, 3, 2)
plt.pcolor(x_1, x_2, u_pred, cmap='jet')
plt.colorbar()
plt.xlabel(r'$x_1$', fontsize=18)
plt.ylabel(r'$x_2$', fontsize=18)
plt.title('Predicted $\hat u(x_1,x_2)$', fontsize=15)

# Error
plt.subplot(1, 3, 3)
plt.pcolor(x_1, x_2, np.abs(usol - u_pred), cmap='jet')
plt.colorbar()
plt.xlabel(r'$x_1$', fontsize=18)
plt.ylabel(r'$x_2$', fontsize=18)
plt.title(r'Absolute error $|u(x_1,x_2)- \hat u(x_1,x_2)|$', fontsize=15)
plt.tight_layout()

# fig.savefig('Helmholtz_non_stiff.png', dpi = 500, bbox_inches='tight')